# Fresnel Wavefront
The `dLux` module offers fully fledged Fresnel optics capabilities based on those in `poppy`. This example shows of how to use the related capabilities.

In [1]:
from dLux import GaussianWavefront, GaussianPropagator
from dLux import CircularAperture as DCircularAperture
from poppy import FresnelWavefront
from poppy import CircularAperture as PCircularAperture
from matplotlib import pyplot
from astropy.units import meter

The first thing to notice is that in `dLux` the `Wavefront` is essentially split over two classes. The reason for this is that the autodiff package we use, `equinox` works by differentiating with respect to class attributes. While we could mix autodiff implementations and use `jax.grad` to take derivatives with respect to function arguments a uniform interface was considered better. 

As a result a `GaussianPropagator` is instantiated whenever we want to take a derivative with respect to a `distance` which is stored as an attribute of the propagator. This means that one `Wavefront` may pass through many propagators. 

A general principle of `dLux` is to abstract mathematically as well as physically, so that the development of physically informed neural network models is easy. The way that this works is the `GaussianPropagator` essentially acts as a differentiable network layer. 

Let's start by using poppy to Fresnel propagate a wavefront through a circular aperture. The parameters I use here come from [this](https://github.com/spacetelescope/poppy/blob/develop/notebooks/Fresnel_Propagation_Demo.ipynb) `poppy` example notebook.

In [2]:
p_wavefront = FresnelWavefront(0.5 * meter, wavelength=2200e-9, npix=1008, oversample=4)

In [3]:
p_wavefront *= PCircularAperture(radius=0.5) 
p_wavefront.propagate_fresnel(5000 * meter)

In [ ]:
pyplot.figure(figsize=(10, 5))
p_wavefront.display("both", colorbar=True, showpadding=True)
pyplot.show()

Now let's replicate that using `dLux` and it's propagators. Now internally `dLux` nests `PyTree`s and this is something that I am really looking to remove in future versions. But for now we wrap the wavefront in a dictionary to pass it between layers. 

Another feature(?) of `dLux` at this level is that a lot less is hidden. At present there is another legacy feature the `CreateWavefront` `Layer` that doubles as a costructor for the `Wavefront`. This is also likely to change in future versions as `Source` and `Scene` classes are created. 

This means that the only parameters required to create the wavefront are `offset` and `wavelength`. The offset is the tilt of the wavefront with respect to the plane of (in this case) the aperture, and has angular units. 

A major difference between `dLux` and `poppy` is that `dLux` doesn't care about units. Everything is in SI units anything else and the code will return incorrect results. This means that all angles are in radians and all distances are in meters with very few exceptions. 

The code is documented with the units so run `help(dLux.Class.method)` to see the units of the parameters. For example, `width : float, meters`, specifies first the type and then the units. 

In [ ]:
import dLux

In [ ]:
npix = 1008
radius = 0.5
offset = [0., 0.]
wavelength = 2200e-07
distance = 0.5

In [ ]:
wavefront = dLux.GaussianWavefront(offset, wavelength, radius)

In [ ]:
wavefront.get_phase_radius()

In [ ]:
p_wavefront.r_c()

In [ ]:
wavefront.get_beam_radius()

In [ ]:
p_wavefront.spot_radius()

In [ ]:
wavefront.waist_radius()

In [ ]:
p_wavefront.w_0

In [ ]:
wavefront.rayleigh_distance()

In [ ]:
wavefront.location_of_waist

In [ ]:
wavefront = dLux.CreateWavefront(npix, radius * 2)({"Wavefront" : wavefront})["Wavefront"]

In [ ]:
wavefront = wavefront.paraxial_interpolate(radius / npix / 4, npix * 4)

In [ ]:
wavefront = dLux.CircularAperture(4 * npix)({"Wavefront" : wavefront})["Wavefront"]

In [ ]:
test = dLux.GaussianPropagator(5000.)({"Wavefront" : wavefront})["Wavefront"]

In [ ]:
pyplot.imshow(test.phase)

So I need to add support for oversampling and I need to make sure that the phase radius is getting calculated at the correct points. 

Do I need to use the phase radius at from or at to?

In [ ]:
help(dLux.GaussianWavefront)

In [ ]:
wavefront = dLux.GaussianPropagator(distance)(
    dLux.CircularAperture(npix)(
        dLux.CreateWavefront(npix, 2 * radius)(
            {"Wavefront" : wavefront})))["Wavefront"]

In [ ]:
pyplot.figure(figsize=(8, 8))
pyplot.imshow(wavefront.amplitude)
pyplot.colorbar()

In [ ]:
help(GaussianPropagator)